In [1]:
# 이미지 처리 분야에서 가장 유명한 신경망 모델인 CNN 을 이용하여 더 높은 인식률을 만들어봅니다.
import tensorflow as tf
from PIL import Image
import glob
import numpy as np

image_width = 64
image_height = 64
image_label = []
All_Img = []
#label = np.zeros(3)

In [2]:
image_file_list = glob.glob("./lemon/*.jpg")
for img in image_file_list:
    image_label.append(0)
    image = Image.open(img)
    image = image.resize((image_width, image_height))
   # All_Img.append(np.float32(image))
    image = np.asarray(image, dtype=np.float32)
#    image = image.reshape(1, image_width, image_height, 3)    
#    print(image.shape)
    All_Img.append(image)    
#    All_Img.append(np.asarray( image, dtype="float32" ))
#     print("lemon ::", np.float32(image))

image_file_list = glob.glob("./hanlabong/*.jpg")
for img in image_file_list:
    image_label.append(1)
    image = Image.open(img)
    image = image.resize((image_width, image_height))    
    image = np.asarray(image, dtype=np.float32)
#    image = image.reshape(1, image_width, image_height, 3)
#     print(image.shape)
    All_Img.append(image)    
#    All_Img.append(np.float32(image))
#    All_Img.append(np.asarray( image, dtype="float32" ))
#     print("hanlabong ::", np.float32(image))
image_file_list = glob.glob("./tangerine/*.jpg")
for img in image_file_list:
    image_label.append(2)
    image = Image.open(img)
    image = image.resize((image_width, image_height))    
    image = np.asarray(image, dtype=np.float32)
    All_Img.append(image)    
image_file_list = glob.glob("./cherry/*.jpg")
for img in image_file_list:
    image_label.append(3)
    image = Image.open(img).convert('RGB')
    image = image.resize((image_width, image_height))    
    image = np.asarray(image, dtype=np.float32)    
#     image = image.reshape(image_width, image_height, 3)    
    All_Img.append(image)
image_file_list = glob.glob("./raspberry/*.jpg")
for img in image_file_list:
    image_label.append(4)
    image = Image.open(img).convert('RGB')
    image = image.resize((image_width, image_height))  
    image = np.asarray(image, dtype=np.float32)

#     image = image.reshape(image_width, image_height, 3)    
    All_Img.append(image)
image_file_list = glob.glob("./strawberry/*.jpg")
for img in image_file_list:
    image_label.append(5)
    image = Image.open(img).convert('RGB')
    image = image.resize((image_width, image_height))  
    image = np.asarray(image, dtype=np.float32)    
#     image = image.reshape(image_width, image_height, 3)
#    image = image.reshape(1, image_width, image_height, 3)    
    All_Img.append(image)    
#    All_Img.append(np.float32(image))
#    All_Img.append(np.asarray( image, dtype="float32" ))
#     print("tangerine ::", np.float32(image))
label = np.eye(6)[image_label]    
print(All_Img[500:1000])
print(label)
print(label[250])
print(label[500])

[array([[[199., 221., 235.],
        [189., 242., 248.],
        [195., 253., 255.],
        ...,
        [186., 125., 158.],
        [181., 120., 153.],
        [177., 116., 147.]],

       [[ 48.,  80.,  79.],
        [ 98., 138., 137.],
        [160., 200., 200.],
        ...,
        [173., 105., 128.],
        [164.,  94., 122.],
        [163.,  90., 119.]],

       [[181., 185., 186.],
        [ 45.,  70.,  67.],
        [207., 255., 255.],
        ...,
        [174., 122.,  49.],
        [164.,  98.,  82.],
        [131.,  66.,  88.]],

       ...,

       [[158.,  65.,   4.],
        [138.,  42.,   0.],
        [116.,  20.,   0.],
        ...,
        [245., 174.,  46.],
        [245., 178.,  48.],
        [241., 172.,  45.]],

       [[164.,  65.,   0.],
        [145.,  42.,   0.],
        [111.,  24.,   7.],
        ...,
        [243., 169.,  38.],
        [237., 168.,  39.],
        [242., 169.,  40.]],

       [[183.,  83.,   8.],
        [155.,  50.,   5.],
        [125., 

In [ ]:
X = tf.placeholder(tf.float32, [None, image_width, image_height, 3])
Y = tf.placeholder(tf.float32, [None, 6])
keep_prob = tf.placeholder(tf.float32)
print('1')
W1 = tf.Variable(tf.random_normal([3,3,3, 128], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print('2')
W2 = tf.Variable(tf.random_normal([3,3,128, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W3 = tf.Variable(tf.random_normal([3,3,64, 32], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W4 = tf.Variable(tf.random_normal([8*8*32, 32], stddev=0.01))
L4 = tf.reshape(L3, [-1, 8*8*32])
L4 = tf.matmul(L4, W4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, keep_prob)
print('3')
W5 = tf.Variable(tf.random_normal([32,6], stddev=0.01))
model = tf.matmul(L4, W5)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
print('4')
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(100):
    _, cost_val = sess.run([optimizer, cost], feed_dict={X: All_Img, Y: label, keep_prob: 0.7})
    if epoch % 5 == 0:
        print('epoch-', epoch+1, ': ', cost_val)
print('end')


1
2
3
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

4
epoch- 1 :  1.7911061
epoch- 6 :  1.7751784
epoch- 11 :  1.7164856
epoch- 16 :  1.634317


In [ ]:
image_label = []
All_Img = []
image_file_list = glob.glob("./lemon/*.jpg")
for img in image_file_list:
    image_label.append(0)
    image = Image.open(img)
    image = image.resize((image_width, image_height))
   # All_Img.append(np.float32(image))
    image = np.asarray(image, dtype=np.float32)
#    image = image.reshape(1, image_width, image_height, 3)    
#    print(image.shape)
    All_Img.append(image)    
#    All_Img.append(np.asarray( image, dtype="float32" ))
#     print("lemon ::", np.float32(image))

image_file_list = glob.glob("./hanlabong/*.jpg")
for img in image_file_list:
    image_label.append(1)
    image = Image.open(img)
    image = image.resize((image_width, image_height))    
    image = np.asarray(image, dtype=np.float32)
#    image = image.reshape(1, image_width, image_height, 3)
#     print(image.shape)
    All_Img.append(image)    
#    All_Img.append(np.float32(image))
#    All_Img.append(np.asarray( image, dtype="float32" ))
#     print("hanlabong ::", np.float32(image))
image_file_list = glob.glob("./tangerine/*.jpg")
for img in image_file_list:
    image_label.append(2)
    image = Image.open(img)
    image = image.resize((image_width, image_height))    
    image = np.asarray(image, dtype=np.float32)
#    image = image.reshape(1, image_width, image_height, 3)    
#    print(image.shape)
    All_Img.append(image)    
#    All_Img.append(np.float32(image))
#    All_Img.append(np.asarray( image, dtype="float32" ))
#     print("tangerine ::", np.float32(image))
image_file_list = glob.glob("./cherry/*.jpg")
for img in image_file_list:
    image_label.append(3)
    image = Image.open(img)
    image = image.resize((image_width, image_height))    
    image = np.asarray(image, dtype=np.float32)
    All_Img.append(image)    
image_file_list = glob.glob("./raspberry/*.jpg")
for img in image_file_list:
    image_label.append(4)
    image = Image.open(img)
    image = image.resize((image_width, image_height))    
    image = np.asarray(image, dtype=np.float32)
    All_Img.append(image)    
image_file_list = glob.glob("./strawberry/*.jpg")
for img in image_file_list:
    image_label.append(5)
    image = Image.open(img)
    image = image.resize((image_width, image_height))    
    image = np.asarray(image, dtype=np.float32)
    All_Img.append(image)  
    
image_label = np.eye(6)[image_label]    

    
print("Test")

predict = tf.argmax(model, 1)
original = tf.argmax(Y, 1)
is_correct = tf.equal(predict, original)
print('X')
print(sess.run(predict, feed_dict={X: All_Img, keep_prob: 1}))
print('Y')
print(sess.run(original, feed_dict={Y: image_label, keep_prob: 1}))

accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:',sess.run(accuracy, feed_dict={X: All_Img, Y:image_label, keep_prob:1}))